In [1]:
# library
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
% matplotlib inline

import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split

C:\Program Files\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
# read csv data
cwd = os.getcwd()
data_path = os.path.join(os.path.dirname(cwd),"data/").replace('/','\\') # for windows

application_train = pd.read_csv(os.path.join(data_path,"application_train.csv"))
application_test = pd.read_csv(os.path.join(data_path,"application_test.csv"))

print("Raw Train data:",len(application_train))
print("Raw Test data:",len(application_test))

Raw Train data: 307511
Raw Test data: 48744


In [4]:
# try get rid of NULL ?
print("Raw Train data (remove NaN):",len(application_train.dropna()))
print("Raw Test data (remove NaN):",len(application_test.dropna()))

## Too many NaN values...

Raw Train data (remove NaN): 8602
Raw Test data (remove NaN): 1739


In [6]:
# rank columns by number of NaN contains
print("Train data")
print(application_train.isnull().astype(int).sum().sort_values(ascending=False))
print("\nTest data")
print(application_test.isnull().astype(int).sum().sort_values(ascending=False))

## Many columns contain NaN values more than 50%

Training data
COMMONAREA_MEDI                214865
COMMONAREA_AVG                 214865
COMMONAREA_MODE                214865
NONLIVINGAPARTMENTS_MODE       213514
NONLIVINGAPARTMENTS_MEDI       213514
NONLIVINGAPARTMENTS_AVG        213514
FONDKAPREMONT_MODE             210295
LIVINGAPARTMENTS_MEDI          210199
LIVINGAPARTMENTS_MODE          210199
LIVINGAPARTMENTS_AVG           210199
FLOORSMIN_MEDI                 208642
FLOORSMIN_MODE                 208642
FLOORSMIN_AVG                  208642
YEARS_BUILD_MEDI               204488
YEARS_BUILD_AVG                204488
YEARS_BUILD_MODE               204488
OWN_CAR_AGE                    202929
LANDAREA_MODE                  182590
LANDAREA_AVG                   182590
LANDAREA_MEDI                  182590
BASEMENTAREA_MEDI              179943
BASEMENTAREA_AVG               179943
BASEMENTAREA_MODE              179943
EXT_SOURCE_1                   173378
NONLIVINGAREA_MEDI             169682
NONLIVINGAREA_AVG              16968

In [34]:
# Get rid of columns that contains NaN values more than 10% , and clear other NaN rows
application_train_fix = application_train.loc[:,application_train.isnull().astype(int).sum()<len(application_train)*0.05].dropna()
application_test_fix = application_test.loc[:,application_train.isnull().astype(int).sum()<len(application_train)*0.05].dropna()

In [35]:
print("Train data:",len(application_train_fix))
print("Test data:",len(application_test_fix))

Train data: 304531
Test data: 47772


In [39]:
# Look at features and dtypes
print(application_train_fix.columns)
print(application_train_fix.dtypes)

Index(['SK_ID_CURR', 'TARGET', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL',
       'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE',
       'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED',
       'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'FLAG_MOBIL', 'FLAG_EMP_PHONE',
       'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL',
       'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT',
       'REGION_RATING_CLIENT_W_CITY', 'HOUR_APPR_PROCESS_START',
       'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION',
       'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY',
       'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY', 'EXT_SOURCE_2',
       'OBS_30_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE',
       'OBS_60_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE',
       'DAYS_LAST_PHONE_CHANGE', 'FLAG_DOCUMENT_2', 'FLAG_DOCUMENT_3',
       'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_5', 'FLAG_DOCUMENT_6',
       'FLAG_DOCUMENT_7', 'FLAG_DOCUMENT_8', 'FLAG_DOCUMENT_9',
       'FL

In [38]:
# Convert to categorical (dummy)
application_train_fix["TARGET"] = application_train_fix["TARGET"].astype('category')
object_to_categorical = [col for col in application_train_fix.columns[application_train_fix.dtypes == object]]
#for col in object_to_categorical:
#    application_train_fix = pd.concat([application_train_fix,pd.get_dummies(application_train_fix[col])])
#    application_test_fix = pd.concat([application_test_fix,pd.get_dummies(application_test_fix[col])])

application_train_fix = application_train_fix.drop(object_to_categorical,axis=1)
application_test_fix = application_test_fix.drop(object_to_categorical,axis=1)

In [40]:
# create dataset
X = application_train_fix.drop(["SK_ID_CURR","TARGET"],axis=1)
Y = application_train_fix["TARGET"]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [41]:
# Random Forest
clf = RandomForestClassifier(max_depth=3, random_state=42)
clf.fit(X_train,Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=3, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=42,
            verbose=0, warm_start=False)

In [43]:
# Features impotance
print(clf.feature_importances_)

[  0.00000000e+00   6.59844484e-03   3.26519445e-02   3.17791943e-02
   4.21111387e-02   1.75277322e-03   1.28240301e-01   2.31463620e-02
   3.59370996e-03   1.07700764e-02   0.00000000e+00   1.62602758e-03
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   9.84687735e-03   1.60833762e-02   2.02017132e-03
   0.00000000e+00   0.00000000e+00   0.00000000e+00   3.50544429e-03
   6.82821466e-03   0.00000000e+00   6.20452826e-01   3.87031592e-04
   3.28557042e-03   0.00000000e+00   0.00000000e+00   4.42686505e-02
   0.00000000e+00   1.10518661e-02   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]


In [46]:
# accuracy by model
print(clf.score(X_test,Y_test))

0.918104651354


In [54]:
# accuracy by random guessing
print(len(list(filter(lambda x: x==0, Y_test)))/len(Y_test))

0.9181046513537032
